In [1]:
import ssl
import os

ssl._create_default_https_context = ssl._create_unverified_context

from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging

predictor_bert = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz")

predictor_bilstm = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/openie-model.2020.03.26.tar.gz")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
import json
with open('data/adj_verb_mft.json', 'r') as infile:
    contents = infile.read()
    sentences = json.loads(contents)
print(sentences[42])
print(len(sentences), "sentences")

{'sentence': 'He had a stitched wound.', 'adj': 'stitched'}
126 sentences


### MFT: Adjective-Verb Differentiation.

In [3]:
def pos_adj_verb_inv(predictor, sentences, model_name):
    failures = list()
    model_outputs = []

    for instance in sentences:
        sentence = instance['sentence']
        adjective = instance['adj']
        
        prediction = predictor.predict(sentence)
        
        detected_verbs = {output['verb'] for output in prediction['verbs']}
        
        if adjective in detected_verbs:
            failures.append(1)
            failure = True
        else:
            failures.append(0)
            failure = False

        # Add output to model_outputs
        model_outputs.append({
            'input': {'sentence': sentence, 'adjective': adjective},
            'output': {'prediction': prediction},
            'failure': failure
        })

    # Write model_outputs to a JSON file
    if not os.path.exists("model_outputs"):
        os.makedirs("model_outputs")
    with open(f'model_outputs/pos_adj_verb_inv_{model_name}.json', 'w') as f:
        json.dump(model_outputs, f, indent=4)
                                  
    return (sum(failures) / len(failures))

print("(BERT) MFT adjectives that look like verbs failure rate:", 100 * pos_adj_verb_inv(predictor_bert, sentences, 'BERT'))
print("(BiLSTM) MFT adjectives that look like verbs failure rate:", 100 * pos_adj_verb_inv(predictor_bilstm, sentences, 'BiLSTM'))

(BERT) MFT adjectives that look like verbs failure rate: 47.61904761904761
(BiLSTM) MFT adjectives that look like verbs failure rate: 47.61904761904761


### Very suspicious, are the models using the same mechanism to detect verbs?